In [1]:
include("../src/crustDistribution.jl")
include("../src/config.jl") # constants defined here 
include("../src/utilities.jl")

Loading crust information from file ../data/crustDistribution.jld


areaAverage

In [2]:
using StatGeochem

In [5]:
using MAT

In [7]:
# Read in mat file
ignp = matread("../igncn1.mat")
comp_compat, _ = convert_dabie("../data/kern_dabie_comp.csv")

([54.8 0.9 … 1.6 0.1; 49.9 1.3 … 1.8 0.1; … ; 48.1 1.3 … 1.7 0.3; 57.9 0.5 … 0.7 0.1], Any["D95-10", "D95-11", "D95-12", "D95-13", "D95-16", "D95-19", "D95-2", "D95-22", "D95-24", "D95-25"  …  "D95-6", "D95-7", "D95-9", "FH7", "HL3", "HL9", "HT1", "HT4", "HT6", "TC3"])

In [9]:
n = size(comp_compat)[1]
ign_new = Dict{String,Any}()
for re in RESAMPLED_ELEMENTS
    ign_new[re] = fill(NaN, n)
end
for (i, ce) in enumerate(COMPOSITION_ELEMENTS)
    ign_new[ce] = comp_compat[:,i]
end

ign_new["elements"] = RESAMPLED_ELEMENTS

# Set uncertainties
ign_new["err"] = Dict()
for e in RESAMPLED_ELEMENTS
    ign_new["err"][e] = ign_new[e] .* (ignp["err2srel"][e] / 2) # default
    ign_new["err"][e][isnan.(ign_new[e])] .= nanstd(ign_new[e]) # missing data gets std, which is a much larger uncertainty
end

# make this the usual ign 
ign = ign_new 

Dict{String,Any} with 54 entries:
  "Pd"    => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "MgO"   => [3.9, 7.7, 9.4, 0.5, 2.2, 6.8, 8.2, 1.0, 6.1, 8.5  …  6.6, 2.0, 0.…
  "Nb"    => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "Gd"    => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "Age"   => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "TiO2"  => [0.9, 1.3, 0.8, 0.2, 0.9, 0.9, 1.0, 0.5, 1.3, 0.8  …  1.9, 0.8, 0.…
  "Be"    => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "Sr"    => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "Ga"    => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, Na…
  "Al2O3" => [17.3, 15.6, 12.1, 13.8, 16.9, 14.8, 15.0, 13.9, 15.3, 19.3  …  17…
  "CO2"   => [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1  …  0.3, 0.1, 0.…
  "Y"     => [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN

In [23]:
resampled = bsresample(ign, 100, RESAMPLED_ELEMENTS)

LoadError: [91mMethodError: no method matching getindex(::typeof(in), ::String)[39m

In [17]:
data = unelementify(ign, RESAMPLED_ELEMENTS, floatout=true)

30×52 Array{Float64,2}:
 54.8  0.9  17.3   7.71947    3.9   6.3  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 49.9  1.3  15.6   8.85954    7.7   8.3     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 53.9  0.8  12.1   6.42968    9.4   6.4     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 73.4  0.2  13.8   1.44991    0.5   1.2     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 60.7  0.9  16.9   5.55954    2.2   4.5     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 56.5  0.9  14.8   6.79962    6.8   5.6  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 48.3  1.0  15.0   9.18941    8.2  11.3     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 71.0  0.5  13.9   3.04972    1.0   3.2     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 52.4  1.3  15.3  11.7397     6.1   8.5     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 48.1  0.8  19.3   9.10983    8.5  10.8     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 56.0  1.2  16.2   8.5896     4.3   6.4  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 39.9  0.5   6.8  14.4692    30.3   3.6     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 49.3  0.4   3.7   8.90945   19.3  14.5 

In [65]:
 function dbsresample2(dataset, nrows, elements=dataset["elements"], p=min(0.2,nrows/length(elements[1]));
            kernel = gaussian,
            rng = MersenneTwister()
        )
        # 2d array of nominal values
        println("hello world")
        println(typeof(dataset))
        k = sort(collect(keys(dataset)))
        println(k)
        data = unelementify(dataset, elements, floatout=true)
        #return data

        # 2d array of absolute 1-sigma uncertainties
        if haskey(dataset, "err") && isa(dataset["err"], Dict)
            sigma = unelementify(dataset["err"], elements, floatout=true)
        else
            sigma = unelementify(dataset, elements.*"_sigma", floatout=true)
        end

        # Resample
        sdata = bsresample(data, sigma, nrows, p, kernel=kernel, rng=rng)
        return elementify(sdata, elements, skipstart=0)
    end

function difguy(dataset, nrows, elements)
    println("hello world")
    println(typeof(dataset))
    data = unelementify(dataset, elements, floatout=true)
end



difguy (generic function with 1 method)

In [66]:
resampled = dbsresample2(ign, 100, RESAMPLED_ELEMENTS)

LoadError: [91mUndefVarError: MersenneTwister not defined[39m

In [60]:
resampled

30×52 Array{Float64,2}:
 54.8  0.9  17.3   7.71947    3.9   6.3  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 49.9  1.3  15.6   8.85954    7.7   8.3     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 53.9  0.8  12.1   6.42968    9.4   6.4     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 73.4  0.2  13.8   1.44991    0.5   1.2     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 60.7  0.9  16.9   5.55954    2.2   4.5     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 56.5  0.9  14.8   6.79962    6.8   5.6  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 48.3  1.0  15.0   9.18941    8.2  11.3     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 71.0  0.5  13.9   3.04972    1.0   3.2     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 52.4  1.3  15.3  11.7397     6.1   8.5     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 48.1  0.8  19.3   9.10983    8.5  10.8     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 56.0  1.2  16.2   8.5896     4.3   6.4  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 39.9  0.5   6.8  14.4692    30.3   3.6     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 49.3  0.4   3.7   8.90945   19.3  14.5 